# Tile Generation Benchmarks using CMR

## Explanation

In this notebook we explore tile generation using titiler-cmr with the GPM IMERG dataset.

## Setup

In [ ]:
import earthaccess
import fsspec
import hvplot.pandas  # noqa
import morecantile
import xarray as xr
from cmr_tile_test import CMRTileTest

import helpers.dataframe as dataframe_helpers
import helpers.eodc_hub_role as eodc_hub_role

In [ ]:
doi = "10.5067/GPM/IMERGDE/DAY/07"
concept_id = "C2723754850-GES_DISC"
variable = "precipitation"
cmr_query = {"temporal": ("2024-02-11", "2024-02-11")}

## Determine number of zoom levels to include in the test

In [ ]:
results = earthaccess.search_data(
    concept_id=concept_id, cloud_hosted=True, temporal=cmr_query["temporal"]
)
with fsspec.open(results[0].data_links("direct")[0]) as f:
    ds = xr.open_dataset(f, decode_cf=False)
resolution = max(
    ds.lat.diff(dim="lat").max().values, ds.lon.diff(dim="lon").max().values
)

In [ ]:
tms = morecantile.tms.get("WebMercatorQuad")
max_zoom = tms.zoom_for_res(resolution, max_z=30, zoom_level_strategy="auto")
max_zoom

## Run test

In [ ]:
cmr_test = CMRTileTest(
    dataset_id=concept_id,
    lat_extent=[-85, 85],
    lon_extent=[-180, 180],
    cmr_query=cmr_query,
    variable=variable,
)

In [ ]:
iterations = 1
zoom_levels = 3  # max_zoom
zooms = range(zoom_levels)

In [ ]:
print(f"Generate indexes for {iterations} random tiles for {zoom_levels} zoom levels")
arguments = []
for zoom in zooms:
    arguments.extend(cmr_test.generate_arguments(iterations, zoom))

In [ ]:
print("Run the tile generation tests")
timings = cmr_test.run_batch(arguments=arguments)

## Store results on s3

In [ ]:
credentials = eodc_hub_role.fetch_and_set_credentials()
cmr_test_results = cmr_test.store_results(credentials)

## Load + plot results 

In [ ]:
credentials = eodc_hub_role.fetch_and_set_credentials()
df = dataframe_helpers.load_all_into_dataframe(
    credentials, [cmr_test_results], use_boto3=False
)
df = dataframe_helpers.expand_timings(df)

In [ ]:
df.hvplot.box(
    y="time",
    by=["zoom"],
    ylabel="Time to tile (ms)",
    xlabel="Zoom level",
    legend=False,
)